In [9]:
!pip install modelscope

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 29.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [14]:
from model_download import download_model

download_model()

开始下载 Qwen-Image 模型到: ./models/Qwen-Image


2025-08-21 01:01:13,461 - modelscope - INFO - Got 31 files, start to download ...


Processing 31 items:   0%|          | 0.00/31.0 [00:00<?, ?it/s]

2025-08-21 01:10:28,848 - modelscope - INFO - Download model 'Qwen/Qwen-Image' successfully.


模型下载完成，保存在: ./models/Qwen-Image


In [1]:
!pip install ./diffusers

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Processing ./diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.36.0.dev0-py3-none-any.whl size=4161451 sha256=5e58da35dc366096e1c3d93c99eb120660f420320d07c90bb89299b8d3852f68
  Stored in directory: /tmp/pip-ephem-wheel-cache-rw7dqoeu/wheels/6a/b4/7c/f91bd9d8e0677ddc65c758324e3aa4399f3c5bd64e3f4c3561
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.35.1
    Uninstalling diffusers-0.35.1:
      Successfully uninstalled diffusers-0.35.1

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
!pip install transformers

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 42.0 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [5]:
from importlib.metadata import version
print("diffusers version:", version("diffusers"))
print("torch version:", version("torch"))
print("transformers version:", version("transformers"))

diffusers version: 0.36.0.dev0
torch version: 2.4.0+cu121
transformers version: 4.55.2


In [6]:
# 在 notebook 单元格中运行
!pip install ipywidgets tqdm accelerate

Looking in indexes: https://mirrors.aliyun.com/pypi/simple

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [15]:
import torch

# Load the pipeline
if torch.cuda.is_available():
    torch_dtype = torch.bfloat16
    device = "cuda"
else:
    torch_dtype = torch.float32
    device = "cpu"
print(f"使用的是{device}")

使用的是cuda


In [16]:
from diffusers import QwenImagePipeline

model_name = "./models/Qwen-Image"
pipe = QwenImagePipeline.from_pretrained(model_name, torch_dtype=torch_dtype)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

The config attributes {'pooled_projection_dim': 768} were passed to QwenImageTransformer2DModel, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# from diffusers import DiffusionPipeline


# model_name = "./models/Qwen-Image"


# pipe = DiffusionPipeline.from_pretrained(model_name, torch_dtype=torch_dtype)
# print(f"管道已加载成功{pipe}")

In [17]:
pipe = pipe.to(device)

positive_magic = {
    "en": ", Ultra HD, 4K, cinematic composition.", # for english prompt
    "zh": ", 超清，4K，电影级构图." # for chinese prompt
}

# Generate image
prompt = '''A coffee shop entrance features a chalkboard sign reading "Qwen Coffee 😊 $2 per cup," with a neon light beside it displaying "通义千问". Next to it hangs a poster showing a beautiful Chinese woman, and beneath the poster is written "π≈3.1415926-53589793-23846264-33832795-02384197".'''

negative_prompt = " " # Recommended if you don't use a negative prompt.


# Generate with different aspect ratios
aspect_ratios = {
    "1:1": (1328, 1328),
    "16:9": (1664, 928),
    "9:16": (928, 1664),
    "4:3": (1472, 1104),
    "3:4": (1104, 1472),
    "3:2": (1584, 1056),
    "2:3": (1056, 1584),
}

width, height = aspect_ratios["16:9"]

image = pipe(
    prompt=prompt + positive_magic["en"],
    negative_prompt=negative_prompt,
    width=width,
    height=height,
    num_inference_steps=50,
    true_cfg_scale=4.0,
    generator=torch.Generator(device="cuda").manual_seed(42)
).images[0]

image.save("example.png")

OutOfMemoryError: CUDA out of memory. Tried to allocate 72.00 MiB. GPU 0 has a total capacity of 44.53 GiB of which 35.25 MiB is free. Process 3390128 has 44.49 GiB memory in use. Of the allocated memory 43.88 GiB is allocated by PyTorch, and 198.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)